# Scoring Data Downloading & Preparation

This notebook focusses on the 4 dimensions used for creating the scores for 2001 and 2011, and that we will try to predict for 2021.

In [ ]:
import matplotlib as mpl
mpl.use('TkAgg')
%matplotlib inline
import matplotlib.pyplot as plt 

In [ ]:
# For reproducibility
import random
import numpy as np
r_state = 42
random.seed(r_state) 
np.random.seed(r_state)

In [ ]:
import pandas as pd
import geopandas as gpd
import pysal as ps
import requests
import glob
import re
import os
import io
import zipfile
from io import BytesIO

from scipy.stats import gmean
from geoconvert import geoconvert

lkp = os.path.join('data','lkp')
src = os.path.join('data','src')

canonical = os.path.join('data','canonical')
converted = os.path.join(canonical,'converted')
housing   = os.path.join(canonical,'housing')
household = os.path.join(canonical,'households')
work      = os.path.join(canonical,'work')

for d in [canonical,converted,housing,household,work]:
    if not os.path.exists(d):
        os.makedirs(d)

In [ ]:
# Make sure you always run this!
boroughs = ['City of London','Barking and Dagenham','Barnet','Bexley','Brent','Bromley',
            'Camden','Croydon','Ealing','Enfield','Greenwich','Hackney','Hammersmith and Fulham',
            'Haringey','Harrow','Havering','Hillingdon','Hounslow','Islington',
            'Kensington and Chelsea','Kingston upon Thames','Lambeth','Lewisham',
            'Merton','Newham','Redbridge','Richmond upon Thames','Southwark','Sutton',
            'Tower Hamlets','Waltham Forest','Wandsworth','Westminster']

In [ ]:
ldn2011 = pd.read_pickle(os.path.join(lkp,'LSOAs 2011.pkl'))
ldn2004 = pd.read_pickle(os.path.join(lkp,'LSOAs 2004.pkl'))

print("Have built London LSOA filter data for use where needed...")
print("\t2004: " + str(ldn2004.shape[0]) + " rows.")
print("\t2011: " + str(ldn2011.shape[0]) + " rows.")

In [ ]:
def convert_to_2011(df,src,dest,nm):
    """
    Converts 2001 data to be compatible with 2011 data -- 
    this is to deal with the boundary changes that happen
    at each Census.
    """
    gc = geoconvert.geoconvert()
    gc.auto_2001_to_2011(os.path.join(src,nm))

    for f in glob.glob(re.sub("-\d+\.csv","*",nm)):
        fn = re.sub("-converted","",f)
        print("Moving " + f + " to " + converted)
        os.rename(f, os.path.join(converted,fn))
    
    dfc = pd.read_csv(os.path.join(converted,nm), index_col=False)
    
    dfc.columns=df.columns
    
    dfc.to_csv(os.path.join(dest,nm), index=False)
    print("\tConverted file has " + str(dfc.shape[0]) + " rows.")
    print(dfc.sample(2, random_state=r_state))
    return

def get_neighbours(ns, col):
    """
    Find neighbours of a given LSOA.
    """
    neighbours = []
    for n in ns.keys():
        #print(str(n) + " -> " + str(col[n][0][1]))
        neighbours.append(col[n][0][1]) # Not elegant, but column name changes with year
    return neighbours

def get_gmean_from_neighbours(ns, prices):
    """
    Find geometric mean of an LSOAs _neighbours'_ property transactions.
    """
    print("\tSearching for: " + ", ".join(map(str, ns)))
    medians = prices.loc[prices.index.isin(ns),'Median Property Price'].values
    print("\tFound median prices: " + ", ".join(map(str, medians)))
    return round(gmean(medians[np.logical_not(np.isnan(medians))]), -1)

## Modelled LSOA Household Income

In [ ]:
print("Getting LSOA Household Income estimates from London Data Store...")
print("Note: this has already been converted to use LSOA 2011 codes!")
url  = ('https://files.datapress.com/london/dataset/'
        'household-income-estimates-small-areas/'
        'modelled-household-income-estimates-lsoa.csv')

# Retrieve it
hhi  = pd.read_csv(url, encoding='latin-1')

# Rename key cols
hhi.rename(columns={'Code':'lsoacd'}, inplace=True)

hhi.set_index('lsoacd', inplace=True)

# And break them down into subsets
hhi2001 = hhi.loc[:,['Median 2001/02']]
hhi2011 = hhi.loc[:,['Median 2011/12']]

# Rename the columns
print("Renaming...")
hhi2001.rename(columns=lambda x: x.replace(' 2001/02', ' Income'), inplace=True)
hhi2011.rename(columns=lambda x: x.replace(' 2011/12', ' Income'), inplace=True)

# Convert to numeric
print("Converting to numeric data types...")
for df in [hhi2001, hhi2011]:
    df.loc[:,('Median Income')] = pd.to_numeric(df.loc[:,'Median Income'].str.replace("\D+",""), errors='coerce')

# And save to CSV
hhi2001.to_csv(os.path.join(work,'Income-2001.csv'), index=True, header=True, encoding='utf-8')
hhi2011.to_csv(os.path.join(work,'Income-2011.csv'), index=True, header=True, encoding='utf-8')

# Sanity check
print("Have " + str(hhi2001.shape[0]) + " rows of data.")
print("   Done.")

## Median Housing & Sales

In [ ]:
print("Getting LSOA Housing Value estimates from London Data Store...")
print("Note: this has already been converted to use LSOA 2011 codes!")
url  = ('https://files.datapress.com/london/dataset/'
        'average-house-prices-ward-lsoa-msoa/' 
        '2016-07-06T14:34:00/house-prices-LSOAs.csv')

# Retrieve it
hhv  = pd.read_csv(url, na_values=".", encoding='latin-1')

# Simplify column names
hhv.rename(columns={
        'Lower Super Output Area':'lsoacd',
        'Names':'Name',
        'Census 2011 dwellings':'Dwellings_2011'}, inplace=True)

# Set the index
hhv.set_index('lsoacd', inplace=True)
hhv.rename(columns=lambda x: re.sub('-',' ',re.sub('(?:\\([^\\)]+\\))','',x)), inplace=True)


# And break them down into subsets
hhv2001 = hhv.loc[:,['Median 2001','Sales 1995',
               'Sales 1996', 'Sales 1997', 
               'Sales 1998', 'Sales 1999',
               'Sales 2000', 'Sales 2001']]
hhv2011 = hhv.loc[:,['Median 2011','Sales 2005', 
               'Sales 2006', 'Sales 2007',
               'Sales 2008', 'Sales 2009', 
               'Sales 2010', 'Sales 2011']]

# Rename keys for consistency
hhv2001.rename(columns={'Median 2001':'Median Property Price'}, inplace=True)
hhv2011.rename(columns={'Median 2011':'Median Property Price'}, inplace=True)

# Remove underscores
hhv2001.rename(columns=lambda x: x.replace('_',''), inplace=True)
hhv2011.rename(columns=lambda x: x.replace('_',''), inplace=True)

# Sanity check
print("Have " + str(hhv2001.shape[0]) + " rows of data.")
print("Done.")

### Dealing with NaNs

It should be only the house price data that has NaNs -- I can't be 100% certain, but I'd assume that this is because there were no transactions in these LSOAs that year (and they don't -- and shouldn't -- fill in missing data by looking back at previous years) so there was nothing to report, or because those LSOAs didn't exist and they've not done a good job of back-filling with real data. 

We don't want to simply drop these areas from the analysis since they'll create gaps in our reuslts for no particularly good reason. Looking to the raw Land Registry data and then trying to work out the most representative range of nearby values would work but represents a huge amount of effort for relatively little return. Consequently, the most effective solution appears to me to take the geometric mean of the surrounding medians as a 'best guess' as to what values in the LSOA might be. The geometric mean is more robust to outliers and so should cope fairly well in those areas where there is a steep price gradient. But to make life easy you'll see below what values were used in each calculation.

In [ ]:
# This data has already been mapped on to 
# 2011 LSOA boundaries... For some reason the
# ones from the GLA Data Store don't work, but
# the full one available from the OS do. 
qw       = ps.weights.Queen.from_shapefile(
                os.path.join('data','shp','LSOA-Weights.shp')) # Weights/Adjacency
fh       = ps.open(
                os.path.join('data','shp','LSOA-Weights.dbf'))
cds      = fh.by_col['lsoa11cd'] # LSOA 2011 Census code

In [ ]:
print("2001...")
nan01 = hhv2001[hhv2001['Median Property Price'].isnull()].index.values
print("\tLooking for neighbours of " + str(len(nan01)) + " areas without house prices.")

for z in nan01:
    print("Finding neighbours for " + z + "(id: " + str(cds.index(z)) + ")")
    neighbours01 = get_neighbours(qw[ cds.index(z) ], fh)
    
    m = get_gmean_from_neighbours(neighbours01, hhv2001)
    print("\t" + z + " has been assigned geometric mean of neighbours: " + str(m))
    hhv2001.loc[z,'Median Property Price'] = m
print(" ")

print("2011...")
nan11 = hhv2011[hhv2011['Median Property Price'].isnull()].index.values
print("\tLooking for neighbours of " + str(len(nan11)) + " areas without house prices.")

for z in nan11:
    print("Finding neighbours for " + z)
    neighbours11 = get_neighbours(qw[ cds.index(z) ], fh)
    
    m = get_gmean_from_neighbours(neighbours11, hhv2011)
    print("\t" + z + " has been assigned geometric mean of neighbours: " + str(m))
    hhv2011.loc[z,'Median Property Price'] = m
print(" ")

print("Done.")

In [ ]:
# This should have been pulled from real data
hhv2001[hhv2001.index=='E01001510']

In [ ]:
# This should have been assigned from the geometric mean of neighbours calculation
hhv2011[hhv2011.index=='E01001510']

In [ ]:
# And save to CSV
hhv2001.loc[:,['Median Property Price']].to_csv(os.path.join(housing,'Values-2001.csv'), index=True, header=True, encoding='utf-8')
hhv2011.loc[:,['Median Property Price']].to_csv(os.path.join(housing,'Values-2011.csv'), index=True, header=True, encoding='utf-8')

# Probably not useful but worked out just in case
# the rate of transactions in the runup to the 
# Census year is a useful indicator.
hhv2001.loc[:,['Sales 1995','Sales 1996', 'Sales 1997', 'Sales 1998', 'Sales 1999',
               'Sales 2000', 'Sales 2001']].to_csv(os.path.join(housing,'Transactions-2001.csv'), index=True, header=True, encoding='utf-8')
hhv2011.loc[:,['Sales 2005', 'Sales 2006', 'Sales 2007','Sales 2008', 'Sales 2009', 
               'Sales 2010', 'Sales 2011']].to_csv(os.path.join(housing,'Transactions-2011.csv'), index=True, header=True, encoding='utf-8')

# Sanity check
print("Have " + str(hhv2001.shape[0]) + " rows of data.")
print("   Done.")

## Occupations

### 2001 Data

In [ ]:
print("Processing 2001 Occupations data from Nomis Table KS012a...")
print("Note: this needs to be converted to LSOA 2011 codes using GeoConvert!")

# Load the data from the KS012a table
occ_01 = pd.read_csv(os.path.join(src,"2001","ks012a.csv.gz"),
                      header=5, skip_blank_lines=True, compression='gzip')

# Rename the columns to something easier to work with
occ_01.rename(columns=lambda x: re.sub("^\d+\. ","",x), inplace=True)
occ_01.rename(columns={
    'mnemonic':'lsoacd', 
    'super output areas - lower layer':'LSOANM', 
    'All categories: Occupation':'Total',
    'Managers and senior officials':'Managerial',
    'Professional occupations':'Professional',
    'Associate professional and technical occupations':'Technical',
    'Administrative and secretarial occupations':'Administrative',
    'Skilled trades occupations':'Skilled',
    'Personal service occupations':'Personal Service',
    'Sales and customer service occupations':'Customer Service',
    'Process, plant and machine operatives':'Operators',
    'Elementary occupations':'Elementary'
}, inplace=True)

# Select only those rows that are in the London 2001 LSOA list
occ_01 = occ_01.loc[occ_01.lsoacd.isin(ldn2004.lsoacd.values)]

# Drop the columns we're not interested in
occ_01.drop('LSOANM', axis=1, inplace=True)

occ_01.to_csv(os.path.join(src,"Occupations-2001.csv"), index=False, header=True, encoding='utf-8')

# Sanity check
print("Wrote " + str(occ_01.shape[0]) + " rows to output file.")

# convert_to_2011(df,src,dest,nm)
convert_to_2011(occ_01, src, work, 'Occupations-2001.csv')

# Sanity check
print("Done.")

### 2011 Data

In [ ]:
print("Processing 2011 Occupations data from Nomis Table KS610EW...")
print("Note: this does not need to be converted.")

# Load the data from the KS610EW table
occ_11 = pd.read_csv(os.path.join(src,"2011","ks610ew.csv.gz"),
                      header=7, skip_blank_lines=True, compression='gzip')

# Rename the columns to something easier to work with
occ_11.rename(columns=lambda x: re.sub("^\d+\. ","",x), inplace=True)
occ_11.rename(columns={
    'mnemonic':'lsoacd', 
    '2011 super output area - lower layer':'LSOANM', 
    'All categories: Occupation':'Total',
    'Managers, directors and senior officials':'Managerial',
    'Professional occupations':'Professional',
    'Associate professional and technical occupations':'Technical',
    'Administrative and secretarial occupations':'Administrative',
    'Skilled trades occupations':'Skilled',
    'Caring, leisure and other service occupations':'Personal Service',
    'Sales and customer service occupations':'Customer Service',
    'Process plant and machine operatives':'Operators',
    'Elementary occupations':'Elementary'
}, inplace=True)

# Select only those rows that are in the London 2011 LSOA list
occ_11 = occ_11.loc[occ_11.lsoacd.isin(ldn2011.lsoacd.values)]

# Drop the columns we're not interested in
occ_11.drop('LSOANM', axis=1, inplace=True)
occ_11.to_csv(os.path.join(work,"Occupations-2011.csv"), index=False, header=True, encoding='utf-8')

# Sanity check
print("Wrote " + str(occ_11.shape[0]) + " rows to output file.")
print("Done.")

## Qualifications

### 2001 Data

In [ ]:
print("Processing 2001 Qualifications data from Nomis Table KS013...")
print("Note: this needs to be converted to LSOA 2011 codes using GeoConvert!")

# Load the data from the KS013 table
quals_01 = pd.read_csv(os.path.join(src,"2001","ks013.csv.gz"),
                      header=5, skip_blank_lines=True, compression='gzip')

# Rename the columns to something easier to work with
quals_01.rename(columns=lambda x: re.sub("(?:Highest level of qualification: )(.+) qualifications","\\1",x), inplace=True)
quals_01.rename(columns=lambda x: re.sub("(?:Full-time students: Age 18 to 74: Economically )(?:active: )?(.+)","Students: \\1",x), inplace=True)
quals_01.rename(columns={
    'mnemonic':'lsoacd', 
    'super output areas - lower layer':'LSOANM', 
    'All people aged 16-74':'Total'}, inplace=True)

# Select only those rows that are in the London 2001 LSOA list
quals_01 = quals_01.loc[quals_01.lsoacd.isin(ldn2004.lsoacd.values)]

# Drop the columns we're not interested in
quals_01.drop('LSOANM', axis=1, inplace=True)

quals_01.to_csv(os.path.join(src,"Qualifications-2001.csv"), index=False, header=True, encoding='utf-8')

# Sanity check
print("Wrote " + str(quals_01.shape[0]) + " rows to output file.")

# convert_to_2011(df,src,dest,nm)
convert_to_2011(quals_01, src, work, 'Qualifications-2001.csv')

# Sanity check
print("Done.")

### 2011 Data

Note that we don't make use of the 'Apprenticeship' column as it has no equivalent in the 2001 data and we need a comparable base.

In [ ]:
print("Processing 2011 Qualifications data from Nomis Table KS501EW...")
print("Note: this does not need to be converted.")

# Load the data from the KS501EW table
quals_11 = pd.read_csv(os.path.join(src,"2011","ks501ew.csv.gz"),
                      header=6, skip_blank_lines=True, compression='gzip')

# Rename the columns to something easier to work with
quals_11.rename(columns=lambda x: re.sub("(?:Highest level of qualification: )(.+) qualifications","\\1",x), inplace=True)
quals_11.rename(columns=lambda x: re.sub("(?:Full-time students: Age 18 to 74: Economically )(?:active: )?(.+)","Students: \\1",x), inplace=True)
quals_11.rename(columns={'mnemonic':'lsoacd', '2011 super output area - lower layer':'LSOANM', 'All categories: Highest level of qualification':'Total'}, inplace=True)

# Select only those rows that are in the London 2011 LSOA list
quals_11 = quals_11.loc[quals_11.lsoacd.isin(ldn2011.lsoacd.values)]

# Drop the columns we're not interested in -- although it
# would be nice to keep the Apprenticeship data we can't
# seemingly compare it to the 2001 data. As far as I can tell
# this is because the question was new in 2011, so presumably
# respondents in 2001 would have been folded into one of the 
# 'lower' qualifications brackets. For a brief analysis, see
# https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/employmentandemployeetypes/articles/qualificationsandlabourmarketparticipationinenglandandwales/2014-06-18
quals_11.drop(['LSOANM','Highest level of qualification: Apprenticeship'], axis=1, inplace=True)

quals_11.to_csv(os.path.join(work,"Qualifications-2011.csv"), index=False, header=True, encoding='utf-8')

# Sanity check
print("Wrote " + str(quals_11.shape[0]) + " rows to output file.")
print("Done.")

## Rents

There is some generic data on rents that might be useful, but unfortunately no one seems to have statistics as far back as 2001 -- the earliest I could find dated to 2014 and the VOA specifically recommends against trying to compare across years with much of their data:

- [VOA Home Page @ National Archives](http://webarchive.nationalarchives.gov.uk/20141002130950/http://www.voa.gov.uk/corporate/index.html)
- [Private Rental Market Stats @ National Archives](http://webarchive.nationalarchives.gov.uk/20141002135606/http://www.voa.gov.uk/corporate/statisticalReleases/110929_PrivateResidentialRentalMarketStatistics.html)
- [General VOA Stats Page @ National Archives](http://webarchive.nationalarchives.gov.uk/20141002132258/http://www.voa.gov.uk/corporate/publications/statistics.html)